In [56]:
import os 
import pandas as pd
from  actual_secret_key import openai_api_key
# from secret_key import openai_api_key 
from tqdm.notebook import tqdm
tqdm.pandas()
from langchain.embeddings import OpenAIEmbeddings

In [2]:
os.environ['OPENAI_API_KEY'] = openai_api_key

In [3]:
embeddings_generator = OpenAIEmbeddings()

In [4]:
embeddings_generator.model

'text-embedding-ada-002'

OpenAI Model [text-embedding-ada-002](https://platform.openai.com/docs/models/embeddings) selected for good Clustering score on [Massive Text Embedding Benchmark (MTEB)](https://huggingface.co/spaces/mteb/leaderboard).

In [5]:
articles_df = pd.read_csv('data/article_titles.csv')
articles_df.head()

,Unnamed: 0,Article Number,Article Title,Part,Part Title
0,0,1,Name and territory of the Union.,PART I,THE UNION AND ITS TERRITORY ARTICLES
1,1,2,Admission or establishment of new States. [,PART I,THE UNION AND ITS TERRITORY ARTICLES
2,2,2A,Sikkim to be associated with the Union. —Omitt...,PART I,THE UNION AND ITS TERRITORY ARTICLES
3,3,3,Formation of new States and alteration of area...,PART I,THE UNION AND ITS TERRITORY ARTICLES
4,4,4,Laws made under articles 2 and 3 to provide fo...,PART I,THE UNION AND ITS TERRITORY ARTICLES


In [8]:
sample = "93 Readjustment after each census."
text_embd =  embeddings_generator.embed_query(sample)
text_embd

[-0.0034833423306342326,
 0.01654504730987026,
 0.006847369671463557,
 -0.019223013110075418,
 -0.00028026655805019166,
 0.015709840984252103,
 -0.01525909484632965,
 -0.014874634658633944,
 -0.02327973114534511,
 -0.008650355620137176,
 -0.003463456499000081,
 0.01300536229407231,
 0.008809442273210387,
 -0.014410631144401633,
 -0.009677792505264456,
 -0.006907027399196645,
 0.01818894707113194,
 -0.018838551991057176,
 -0.004434550000471737,
 -0.018759009595843108,
 -0.01809614636828548,
 0.01510000819325644,
 -0.001073837236550511,
 -0.0006893769906471471,
 0.0062342218378985265,
 -0.006201078862785153,
 0.01897112389151067,
 0.0015585554594746316,
 0.0196339871190683,
 0.025758837232224944,
 0.0016521848065481183,
 -0.0010655514927721676,
 -0.010035737940340447,
 -0.0070594858297761956,
 -0.035476400003773896,
 -0.014622746371391736,
 -0.011732666275519017,
 -0.03123408056281128,
 0.007987493323902084,
 -0.01881203723843746,
 0.00791457821985914,
 0.007198686884045889,
 -0.00819960

In [9]:
len(text_embd)

1536

In [11]:
articles_df.head()

,Unnamed: 0,Article Number,Article Title,Part,Part Title
0,0,1,Name and territory of the Union.,PART I,THE UNION AND ITS TERRITORY ARTICLES
1,1,2,Admission or establishment of new States. [,PART I,THE UNION AND ITS TERRITORY ARTICLES
2,2,2A,Sikkim to be associated with the Union. —Omitt...,PART I,THE UNION AND ITS TERRITORY ARTICLES
3,3,3,Formation of new States and alteration of area...,PART I,THE UNION AND ITS TERRITORY ARTICLES
4,4,4,Laws made under articles 2 and 3 to provide fo...,PART I,THE UNION AND ITS TERRITORY ARTICLES


In [54]:
articles_df = articles_df.assign(article_title_word_count = lambda x: x['Article Title'].astype(str).progress_apply(lambda x: len(x.split())))

100%|██████████| 498/498 [00:00<00:00, 238035.71it/s]


In [34]:
articles_df.head()

,Unnamed: 0,Article Number,Article Title,Part,Part Title,article_title_word_count
0,0,1,Name and territory of the Union.,PART I,THE UNION AND ITS TERRITORY ARTICLES,6
1,1,2,Admission or establishment of new States. [,PART I,THE UNION AND ITS TERRITORY ARTICLES,7
2,2,2A,Sikkim to be associated with the Union. —Omitt...,PART I,THE UNION AND ITS TERRITORY ARTICLES,8
3,3,3,Formation of new States and alteration of area...,PART I,THE UNION AND ITS TERRITORY ARTICLES,14
4,4,4,Laws made under articles 2 and 3 to provide fo...,PART I,THE UNION AND ITS TERRITORY ARTICLES,25


In [57]:
%%time
articles_df = articles_df.assign(article_title_embeddings = lambda x: x['Article Title'].astype(str).progress_apply(embeddings_generator.embed_query))

  0%|          | 0/498 [00:00<?, ?it/s]

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID bd81cd9d2829ef9b4f6b4664578e1515 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID bd81cd9d2829ef9b4f6b4664578e1515 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID bd81cd9d2829

CPU times: total: 141 ms
Wall time: 6min 40s


In [60]:
articles_df.shape

(498, 7)

In [66]:
articles_df.drop(articles_df.columns[0], axis=1).to_csv('data/article_title_1kembeddings.csv', header=False)